In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, paddle"""
import deepxde as dde
import numpy as np
a = 1
b = 2
k_r = 1.5
k_i = 1
v_1 = 1
v_2 = 1
v_3 = 1
c=0
def gen_traindata():

    x_lower = -10
    x_upper = 10
    t_lower = -2
    t_upper = 2
    
    x = np.linspace(x_lower, x_upper, 200)
    t = np.linspace(t_lower, t_upper, 200)
    X, T = np.meshgrid(x, t)
    X = np.reshape(X, (-1, 1))
    T = np.reshape(T, (-1, 1))
   
    c = np.log((a * a * v_1 + b * b * v_2 + a * b * v_3 + a * b * v_3) / ((2 * k_r) * (2 * k_r)))
    u1 = (a / 2) * np.exp(-c / 2) * np.cos(k_i*X + (k_r*k_r-k_i*k_i) * T) / np.cosh(k_r* X - 2*k_r*k_i * T + c / 2)
    v1 = (a / 2) * np.exp(-c / 2) * np.sin(k_i*X + (k_r*k_r-k_i*k_i) * T) / np.cosh(k_r* X - 2*k_r*k_i * T + c / 2)
    u2 = (b / 2) * np.exp(-c / 2) * np.cos(k_i*X + (k_r*k_r-k_i*k_i) * T) / np.cosh(k_r* X - 2*k_r*k_i * T + c / 2)
    v2 = (b / 2) * np.exp(-c / 2) * np.sin(k_i*X + (k_r*k_r-k_i*k_i) * T) / np.cosh(k_r* X - 2*k_r*k_i * T + c / 2)

   
    U1=np.reshape(u1,(-1,1))
    V1=np.reshape(v1,(-1,1))
    U2=np.reshape(u2,(-1,1))
    V2=np.reshape(v2,(-1,1))
    
    return np.hstack((X, T)), U1,V1,U2,V2


#kf = dde.Variable(0.05)
#D = dde.Variable(1.0)
m0=dde.Variable(1.5)
m1=dde.Variable(3.0)
m2=dde.Variable(2.0)
m3=dde.Variable(1.0)

def pde(x,y):
    
    u1 = y[:, 0:1]
    v1 = y[:, 1:2]
    u2 = y[:, 2:3]
    v2 = y[:, 3:4]
    

    # 在'jacobian'中，i 是输出分量，j 是输入分量
    u1_t = dde.grad.jacobian(y, x, i=0, j=1)
    v1_t = dde.grad.jacobian(y, x, i=1, j=1)
    u2_t = dde.grad.jacobian(y, x, i=2, j=1)
    v2_t = dde.grad.jacobian(y, x, i=3, j=1)

    u1_x = dde.grad.jacobian(y, x, i=0, j=0)
    v1_x = dde.grad.jacobian(y, x, i=1, j=0)
    u2_x = dde.grad.jacobian(y, x, i=2, j=0)
    v2_x = dde.grad.jacobian(y, x, i=3, j=0)

    # 在“hessian”中，i 和 j 都是输入分量。 （Hessian 原则上可以是 d^2y/dxdt、d^2y/d^2x 等）
    # 输出组件由“组件”选择
    u1_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    v1_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    u2_xx = dde.grad.hessian(y, x, component=2, i=0, j=0)
    v2_xx = dde.grad.hessian(y, x, component=3, i=0, j=0)

    #f_u = u_t + 0.5 * v_xx + (u ** 2 + v ** 2) * v
    #f_v = v_t - 0.5 * u_xx - (u ** 2 + v ** 2) * u

    f_u1 = (
            v1_t + u1_xx
            + 2 * u1 * (m0 * (u1 ** 2 + v1 ** 2) + m1 * (u2 ** 2 + v2 ** 2) + (m2 + m3) * (u1 * u2 + v1 * v2))
           +2*v1*(m2-m3)*(u1*v2-v1*u2)
    )

    f_v1 = (
            u1_t - v1_xx
            - 2 * v1 * (m0 * (u1 ** 2 + v1 ** 2) + m1 * (u2 ** 2 + v2 ** 2) + (m2 + m3) * (u1 * u2 + v1 * v2))
           +2*u1*(m2-m3)*(u1*v2-v1*u2)
    )
    
    f_u2 = (
            v2_t + u2_xx
            + 2 * u2 * (m0 * (u1 ** 2 + v1 ** 2) + m1 * (u2 ** 2 + v2 ** 2) + (m2 + m3) * (u1 * u2 + v1 * v2))
           +2*v2*(m2-m3)*(u1*v2-v1*u2)
    )

    f_v2 = (
           u2_t - v2_xx
            - 2 * v2 * (m0 * (u1 ** 2 + v1 ** 2) + m1 * (u2 ** 2 + v2 ** 2) + (m2 + m3) * (u1 * u2 + v1 * v2))
           +2*u2*(m2-m3)*(u1*v2-v1*u2)
    )
    
    return [f_u1, f_v1,f_u2,f_v2]


def fun_bc(x):
    return x[:, 0:1]


#def fun_init(x):
#    return np.exp(-20 * x[:, 0:1])

def init_cond_u1(x):
    return (a/2) * np.cos(x[:, 0:1]) * np.exp(-c/2) / np.cosh(1.5*x[:, 0:1]+(c/2))
def init_cond_u2(x):
    return (b/2)*np.cos(x[:, 0:1]) * np.exp(-c/2) / np.cosh(1.5*x[:, 0:1]+(c/2))

def init_cond_v1(x):
    return (-a/2) * np.sin(x[:, 0:1]) * np.exp(-c/2) / np.cosh(1.5*x[:, 0:1]+(c/2))

def init_cond_v2(x):
    return (-b/2)*np.sin(x[:, 0:1]) * np.exp(-c/2) / np.cosh(1.5*x[:, 0:1]+(c/2))


#geom = dde.geometry.Interval(0, 1)
#timedomain = dde.geometry.TimeDomain(0, 10)
#geomtime = dde.geometry.GeometryXTime(geom, timedomain)

x_lower = -10
x_upper = 10
t_lower = -2
t_upper = 2

# 创建 2D 域（用于绘图和输入）


# 整个域变平
#X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

# 空间和时间域/几何（对于 deepxde 模型）
geom = dde.geometry.Interval(x_lower, x_upper)
time_domain = dde.geometry.TimeDomain(t_lower, t_upper)
geomtime = dde.geometry.GeometryXTime(geom, time_domain)




# bc_u1_0 = dde.icbc.DirichletBC(
#     geomtime,fun_bc, lambda _, on_boundary: on_boundary, component=0
# )
# bc_u1_1 = dde.icbc.DirichletBC(
#     geomtime, fun_bc, lambda _, on_boundary: on_boundary,  component=0
# )
# bc_v1_0 = dde.icbc.DirichletBC(
#     geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=1
# )
# bc_v1_1 = dde.icbc.DirichletBC(
#     geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=1
# )
# bc_u2_0 = dde.icbc.DirichletBC(
#     geomtime,fun_bc, lambda _, on_boundary: on_boundary,  component=2
# )
# bc_u2_1 = dde.icbc.DirichletBC(
#     geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=2
# )
# bc_v2_0 = dde.icbc.DirichletBC(
#     geomtime,fun_bc, lambda _, on_boundary: on_boundary,component=3
# )
# bc_v2_1 = dde.icbc.DirichletBC(
#     geomtime, fun_bc, lambda _, on_boundary: on_boundary,  component=3
# )

bc_u1_0=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=0
)
bc_v1_0=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=1
)
bc_u2_0=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=2
)
bc_v2_0=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=3
)
bc_u1_1=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=0
)
bc_v1_1=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=1
)
bc_u2_1=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=2
)
bc_v2_1=dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=3
)
#ic1 = dde.icbc.IC(geomtime, fun_init, lambda _, on_initial: on_initial, component=0)
#ic2 = dde.icbc.IC(geomtime, fun_init, lambda _, on_initial: on_initial, component=1)

ic_u1 = dde.icbc.IC(geomtime, init_cond_u1, lambda _, on_initial: on_initial, component=0)
ic_v1 = dde.icbc.IC(geomtime, init_cond_v1, lambda _, on_initial: on_initial, component=1)
ic_u2 = dde.icbc.IC(geomtime, init_cond_u2, lambda _, on_initial: on_initial, component=2)
ic_v2 = dde.icbc.IC(geomtime, init_cond_v2, lambda _, on_initial: on_initial, component=3)

observe_x, u1,v1,u2,v2 = gen_traindata()
observe_y1 = dde.icbc.PointSetBC(observe_x, u1, component=0)
observe_y2 = dde.icbc.PointSetBC(observe_x, v1, component=1)
observe_y3 = dde.icbc.PointSetBC(observe_x, u2, component=2)
observe_y4 = dde.icbc.PointSetBC(observe_x, v2, component=3)

data = dde.data.TimePDE(
    geomtime,
    pde,
    #[bc_a, bc_b, ic1, ic2, observe_y1, observe_y2],
    [bc_u1_0, bc_u1_1, bc_v1_0, bc_v1_1,bc_u2_0, bc_u2_1, bc_v2_0, bc_v2_1, ic_u1, ic_v1,ic_u2, ic_v2,observe_y1, observe_y2,observe_y3,observe_y4],
    num_domain=40000,
    num_boundary=200,
    num_initial=200,
    anchors=observe_x,
    num_test=50000,
)
net = dde.nn.FNN([2] + [20] * 3 + [4], "sin", "Glorot uniform")



model = dde.Model(data, net)
model.compile("adam", lr=0.001, external_trainable_variables=[m0,m1,m2,m3])
variable = dde.callbacks.VariableValue([m0,m1,m2,m3], period=1000, filename="variables.dat")
losshistory, train_state = model.train(iterations=20000, callbacks=[variable])
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
Building feed-forward neural network...
'build' took 0.023654 s

'compile' took 1.253684 s

Initializing variables...
Training model...

0         [3.93e+02, 1.14e+02, 3.91e+02, 3.17e+01, 5.19e-01, 2.31e-02, 4.98e-01, 2.93e-02, 1.34e+00, 3.66e-02, 1.20e-01, 1.56e-03, 1.07e+00, 1.09e+00, 1.64e+00, 5.56e-02, 1.24e+00, 1.21e+00, 1.83e+00, 3.96e-02]    [3.96e+02, 1.15e+02, 3.94e+02, 3.20e+01, 5.19e-01, 2.31e-02, 4.98e-01, 2.93e-02, 1.34e+00, 3.66e-02, 1.20e-01, 1.56e-03, 1.07e+00, 1.09e+00, 1.64e+00, 5.56e-02, 1.24e+00, 1.21e+00, 1.83e+00, 3.96e-02]    []  
1000      [3.94e-03, 5.08e-03, 4.17e-03, 6.93e-03, 2.00e-04, 3.84e-04, 2.03e-03, 4.22e-04, 3.19e-04, 3.35e-04, 9.96e-04, 4.11e-04, 8.94e-03, 8.50e-03, 3.20e-02, 1.72e-02, 1.12e-02, 2.40e-02, 4.11e-02, 3.73e-02]    [3.97e-03, 5.10e-03, 4.14e-03, 6.84e-03, 2.00e-04, 3.84e-04, 2.03e-03, 4.22e-04, 3.19e-04, 3.35e-04, 9.96e-04, 4.11e-04, 8.94e-03, 8.50e-03, 3.20e-02, 1.72e-02, 1.12e-02, 2.40e-02, 4.11e-02, 3.73e-02]    [] 

19000     [1.85e-03, 1.26e-03, 6.99e-03, 4.99e-03, 9.81e-07, 2.15e-06, 2.00e-06, 4.71e-07, 4.18e-06, 3.89e-06, 2.15e-06, 1.87e-06, 8.52e-04, 5.33e-04, 3.04e-03, 1.74e-03, 5.92e-03, 5.12e-03, 2.42e-02, 2.00e-02]    [1.79e-03, 1.20e-03, 6.81e-03, 4.77e-03, 9.81e-07, 2.15e-06, 2.00e-06, 4.71e-07, 4.18e-06, 3.89e-06, 2.15e-06, 1.87e-06, 8.52e-04, 5.33e-04, 3.04e-03, 1.74e-03, 5.92e-03, 5.12e-03, 2.42e-02, 2.00e-02]    []  


In [ ]:
# print(m0)
# print(m1)
# print(m2)
# print(m3)
# np.narray(m0)

In [ ]:
# M=[m0,m1,m2,m3]
# import tensorflow as tf
# for i in range(4):
#     with tf.compat.v1.Session() as sess:
#         sess.run(M[i].initializer)
#         print(sess.run(M[i]))

In [ ]:
import re
import matplotlib.pyplot as plt
lines = open("variables.dat", "r").readlines()
# read output data in fnamevar
Mhat = np.array(
    [
        np.fromstring(
            min(re.findall(re.escape("[") + "(.*?)" + re.escape("]"), line), key=len),
            sep=",",
        )
        for line in lines
    ]
)
l, c = Mhat.shape
plt.semilogy(range(0, l * 100, 100), Mhat[:, 0], "r-")
plt.semilogy(range(0, l * 100, 100), Mhat[:, 1], "k-")
plt.semilogy(range(0, l * 100, 100), Mhat[:, 2], "b-")
plt.semilogy(range(0, l * 100, 100), Mhat[:, 3], "g-")
plt.semilogy(range(0, l * 100, 100), np.ones(Mhat[:, 0].shape) * 1, "r--")
plt.semilogy(range(0, l * 100, 100), np.ones(Mhat[:, 1].shape) * 1, "k--")
plt.semilogy(range(0, l * 100, 100), np.ones(Mhat[:, 2].shape) * 1, "b--")
plt.semilogy(range(0, l * 100, 100), np.ones(Mhat[:, 3].shape) * 1, "g--")
plt.legend(["M1hat", "M2hat", "M3hat", "M4hat","True M1", "True M2","True M3", "True M4"], loc="right")
plt.xlabel("Epochs")
plt.title("Variables")
plt.show()
